## Imports

The Python standard library alone contains extensive functionality. However, some tasks require the use of third-party libraries.

Here is the list of the most common libraries used for scientific and mathematical computing with Python:

   * <font style="font-weight:bold">Matplotlib</font> -> used for graphs visualization [doc](https://matplotlib.org/).
   * <font style="font-weight:bold">Numpy</font> -> fast way to work with arrays [doc](https://docs.scipy.org/doc/numpy/user/quickstart.html).
   * <font style="font-weight:bold">Scipy</font> -> extensions to the functionality of NumPy [doc](https://docs.scipy.org/doc/scipy/reference/).
   
   
Note: If you are using [Anaconda](https://www.anaconda.com/) you already have those packeges installed. However, if you prefer to use your own enviroment you can use pip to install the packges [install with pip](https://packaging.python.org/tutorials/installing-packages/).

### Special package used in this course:

   * <font style="font-weight:bold">Obspy</font> -> Python framework for processing seismological data [doc](https://github.com/obspy/obspy/wiki#getting-started).

<font style="font-weight:bold">math</font> is a built-in python library. 

In this exemple we imported the number <font style="font-weight:bold">pi</font> from it.

In [1]:
from math import pi

print("Pi =", pi)

Pi = 3.141592653589793


We could also import the whole library and use  <font style="font-weight:bold">pi</font> or some other function from it like:

In [2]:
import math

print("Pi =", math.pi)
print("sin(pi/2) =", math.sin(math.pi / 2))

Pi = 3.141592653589793
sin(pi/2) = 1.0


Let's now use the <font style="font-weight:bold">numpy</font> library.

    Note: If you get an import error is because you don't have it installed!!
    
The keywork <font color="green" style="font-weight:bold">as</font> allow us to use the given library (numpy in this case) with an abreviation of it. For Numpy coomun abreviation are: np or num
The keyword <font color="green" style="font-weight:bold">as</font> allowing us to use the given library (numpy in this case) with an abbreviation of it. For Numpy commun abbreviations are <font style="font-weight:bold">np</font> or <font style="font-weight:bold">num</font>.

In [3]:
import numpy as np 

numpy_array = np.linspace(start = 0, stop = 10, num = 6)
print("My first numpy array = ", numpy_array)

list_a = [1,2]
list_b = [0,2]

np_array_a = np.array(list_a) # convert list to array
np_array_b = np.array(list_b) # convert list to array

a_prod_b = np_array_a * np_array_b
a_dot_b = np.dot(np_array_a, np_array_b)
print(" A * B = ", a_prod_b)
print(" A . B = ", a_dot_b)

My first numpy array =  [ 0.  2.  4.  6.  8. 10.]
 A * B =  [0 4]
 A . B =  4


We can also write our own python file and reuse it using <font color="green" style="font-weight:bold">import</font> .

    Note: This is actually a good practice to keep your program clear and easy to read, logically grouping your functions in different files and not write all in one single file.
    
The file must be saved with the extension <font style="font-weight:bold">.py</font>.

In [4]:
# This is our earth.py file.
with open('python_code/earth.py') as file:
    code = file.read()
    print(code)

import math
from typing import NamedTuple

# Gravitational constant in m^3 kg^-1 s^-2
gravitational_constat = 6.67408E-11
mass = 5.972E24 # kg
radius = 6371E3 # m

def get_grativity():
    '''
    Computes the Earth's  gravitational acceleration.
    return: 
    The Earth's  gravitational acceleration.
    '''
    g = - gravitational_constat*mass / (radius * radius)
    return g

def get_weight(mass):
    '''
    Computes the force due to a given mass.
    @param mass: The mass to compute the weight. Mass should be a number.
    return: The force of gravity over the mass.
	F = mass * g
    '''
    weight = mass * get_grativity()
    return weight

# python class structure 
class OrbitalParameters(NamedTuple):
    '''
    Data structure for the orbital parameters.
    '''
    radius: float
    theta: float
    angular_velocity: float
    radial_velocity: float

    def __repr__(self):
      return "OrbitalParameters({},{},{},{}".format(self.radius,self.theta,
                          

In [5]:
from python_code import earth

print("G = ", earth.gravitational_constat, "m^3 kg^-1 s^-2")
print("Earth radius = ", earth.radius, "m")
print("Earth mass = ", earth.mass, "kg")
print("Earth gravity = ", earth.get_grativity(), "m s^-2")
print("My weight = ", earth.get_weight(60), "N")

G =  6.67408e-11 m^3 kg^-1 s^-2
Earth radius =  6371000.0 m
Earth mass =  5.972e+24 kg
Earth gravity =  -9.819649737724955 m s^-2
My weight =  -589.1789842634973 N


###  Orbital solver

In the example below we used some of the concepts we have learned so far, in a more interesting application.

 * Use of functions. 
 * Working with lists.
 * Imports from third-party libraries and our own.
 
#### Extra: 
   At the solve_motion_equation we make use of [yield](https://pythontips.com/2013/09/29/the-python-yield-keyword-explained/)
   
The method solve_motion_equation in the earth file solves a 2-Body motion equation for a radial force: 

$F \propto -\frac{1}{r^2}$, 

The set of equations, in polar coordenates are:

$\begin{align*}
r_{i+1} &= r_i + v_{r_i}dt, & \theta_{i+1} &= \theta_i + \omega_idt, \\
v_{r_{i+1}} &= v_{r_i} + \left( r_i\omega_i^2 - \frac{1.}{r_i^2} \right) dt, &
v_{\theta_{i+1}} &= v_{\theta_i}  \left( 1 - \frac{v_{r_i} dt}{r_i}\right)
\end{align*}$ 

where, $v_{\theta_i} = \omega_i r_i$

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt 
from ipywidgets import widgets

# this is in our local directory.
from python_code import earth 

# global 
orbital_data: [earth.OrbitalParameters]
interactive_plot = None

def update_plot():
    global interactive_plot
    
    if interactive_plot:
        interactive_plot.update(0)

def start_numerical_integration(radius=0.2, angle=90, vr=2):
    global orbital_data
    
    # Get solver from earth. Ps: solver is a generator (yield the values.)
    solver = earth.solve_motion_equation(radius, angle * math.pi / 180., vr)
    orbital_data = []
    
    # loop over solver and store data.
    for op in solver:
        orbital_data.append(op)
    
    # update plot if this function is called.
    update_plot()  

def plot_orbit(step):
    global orbital_data
    
    # setup plot.
    data_size = len(orbital_data)
    max_radius = max([op.radius for op in orbital_data])
    plt.figure(2)
    plt.axis('scaled')
    plt.xlim(- 1.1 * max_radius,  1.1 * max_radius)
    plt.ylim(- 1.1 * max_radius,  1.1 * max_radius)
    plt.xlabel('x')
    plt.ylabel('y')
    
    # index position in the orbital data.
    position_index = step - 1

    # orbital parameters at this step.
    r = orbital_data[position_index].radius
    theta = orbital_data[position_index].theta
    vc = orbital_data[position_index].angular_velocity
    vr = orbital_data[position_index].radial_velocity
    
    # buffering data to a list using list slice.
    trajector_buffer_x = [op.radius * np.cos(op.theta) for op in orbital_data[0:position_index]]
    trajector_buffer_y = [op.radius * np.sin(op.theta) for op in orbital_data[0:position_index]]

    # convert to cartesian coords.
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    # plot radius
    plt.plot([0,x],[0,y], color="black")
    # plot central body
    plt.plot(0, 0, 'o', color="blue", markersize = 20)
    # plot trajectroy
    plt.plot(trajector_buffer_x,trajector_buffer_y, "--", color = "black")
    # plot particle
    plt.plot(x, y, 'o', color="green", markersize = 5)
    
    # plot velocities vectors.
    v = math.sqrt(vc**2 + vr**2)
    vector_scale_c = 5*v/abs(vc + 0.0001)
    vector_scale_r = 5*v/abs(vr + 0.0001)
    sig = np.sign(vr)
    plt.quiver([x],[y], [y/r], [-x/r] ,color = 'r', scale = vector_scale_c)
    plt.quiver([x],[y], [sig*x/r], [sig*y/r] ,color = 'b', scale = vector_scale_r)


Here we have made use of some interactive [widgets](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Events.html) for jupyter notebook.

You can interact with different initial conditions and see what happens with the orbit.

    Note: The units here are arbitrary since we made 
$GM = 1$

In [7]:
caption = widgets.Label(value='Choose initial conditions.')
radius_widget = widgets.FloatSlider(min = 0.1, max = 2., step=0.1)
angle_widget = widgets.FloatSlider(min = 0, max = 360, step=5, value = 90)
vr_widget = widgets.FloatSlider(min = -2, max = 2, step=0.1, value = 0.)
widget_solver = widgets.interactive(start_numerical_integration, radius=radius_widget, 
                                                              angle=angle_widget,vr=vr_widget)
display(caption, widget_solver)

Label(value='Choose initial conditions.')

interactive(children=(FloatSlider(value=0.1, description='radius', max=2.0, min=0.1), FloatSlider(value=90.0, …

Plot the orbit using interaction.

Move the slide bar <font style="font-weight:bold">step</font> to evolve the orbit.

In [8]:
def print_info(change):
    position_index = change['new'] - 1
    r = orbital_data[position_index].radius
    print("Radius = {:.3f}".format(r), end='\r')

def display_plot():
    global interactive_plot
    step_widget = widgets.IntSlider(min=1, max=len(orbital_data), step=10)
    interactive_plot = widgets.interactive(plot_orbit, step = step_widget)
    info = step_widget.observe(print_info, names='value')
    output = interactive_plot.children[-1]
    output.layout.height = '350px'
    return display(interactive_plot)

display_plot()

interactive(children=(IntSlider(value=1, description='step', max=10000, min=1, step=10), Output(layout=Layout(…